In [2]:
import os
import numpy as np
import spacy as sp
import pandas as pd
from bs4 import BeautifulSoup
from spacy import displacy
from multiprocessing import Pool
    
#The 'xx' dataset is the biggest multilanguage one.  It catches the most names
#The 'en' dataset does the best job of parsing organizations and labels verbs and other parts of speech
nlp = sp.load('en')


ModuleNotFoundError: No module named 'spacy'

In [48]:
#commented out to keep code we may want to use later
if False: '''
sample = '<ack><title>Acknowledgements</title><p>The authors wish to acknowledge Diya Ma, Matthew-Lun Wong, Ka-Long Ko, Ka-Hei Ko and Jin-Peng Lee for their important contributions to the software development.</p><sec id=""FPar1""><title>Funding</title><p id=""Par28"">The work described in this paper was supported by a grant from the Research Grants Council of the Hong Kong Special Administrative Region, China (Project No.: CUHK 14113214), grants from the Innovation and Technology Commission (Project No: ITS/149/14FP, GHP/028/14SZ, ITS/293/14FP), grants from CUHK Technology and Business Development Fund (Project No.: TBF16MED002, TBF16MED004), a grant from The Science, Technology and Innovation Commission of Shenzhen Municipality (Project No.: CXZZ20140606164105361), and a grant from The Scientific Research Project of Guangdong Province (Project No.: 2014B090901055).</p></sec></ack>'
sample2 = '<ack id=""ack0010""><title>Acknowledgements</title><p>The authors thank Dr. R Kaneko for the gift of the iSip2 vector; and Mss. T Honma, K Harada, A Morita, and Y Shimoda for providing technical and secretarial assistance. We thank the staff at the Department of Genetic and Behavioral Neuroscience and Bioresource Center, Gunma University Graduate School of Medicine for their critical comments and technical assistance. This study was supported by <funding-source id=""gs1"">Grants-in-Aid for Scientific Research</funding-source> (23115503, 26290002, 15H01415 and 15H05872 to Y.Y.), a Grant-in-Aid for Scientific Research on Innovative Areas (Comprehensive Brain Science Network) (to Y.Y.) from the <funding-source id=""gs2"">Ministry of Education, Culture, Sports, Science and Technology (MEXT)</funding-source> of Japan, a grant from the Co-operative Study Program of the <funding-source id=""gs3"">National Institute for Physiological Sciences</funding-source>, Japan (to Y.Y.), and a grant from the <funding-source id=""gs4"">Takeda Science Foundation</funding-source> (to Y.Y.).</p></ack>'
sample = u'<ack id=""ack0005""><title>Acknowledgments</title><p>The project was supported by a start-up funding provided to the author by the <funding-source id=""gs0005"">Department of Neurology of the University of Utah</funding-source>.</p><p>This project was inspired by studying the work of Dr. Ed Dudek and the results of the initial experiments were discussed with him.</p><p>I am also grateful to Dr. Erika Scholl for her assistance in measuring rat serum osmolarity and to Dr. Noel Carlson for his insightful comments on the manuscript.</p></ack>'

soup = BeautifulSoup(sample,'lxml')

samp_txt = soup.find_all('ack')[0].get_text(separator=' ')

print(samp_txt)

doc = nlp(samp_txt)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)'''

In [49]:
#print('works')
#for token in doc:
#    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#          token.shape_, token.is_alpha, token.is_stop)

In [53]:
files = ['A-B','C-H','I-N','O-Z']

for f in files:    
    df = pd.read_csv(''.join(['~/HackathonData/extracted/articles.',f,'.xml.tar.gz.csv']))

    newdf = pd.DataFrame(df['filename'][:10])
    #print([r for r in newdf.iterrows()])
    newdf.insert(1, 'Names', ['' for r in newdf.iterrows()])
    newdf.insert(2, 'Organizations', ['' for r in newdf.iterrows()])
    newdf.insert(3, 'Verbs', ['' for r in newdf.iterrows()])
    newdf.insert(4, 'Nouns', ['' for r in newdf.iterrows()])
    
    namestrs=[]
    for erow,row in enumerate(df['Acknowledgment_Tag'][:10]):
        if erow%1000 == 0: print("File",f,erow)
        if not isinstance(row,str): row = '<ack></ack>'
        soup = BeautifulSoup(row,'lxml')

        for ele in soup.find_all('title'):
            ele.decompose()
        samp_txt = soup.find_all('ack')[0].get_text(separator=' ')

        doc = nlp(samp_txt)
        
        verbstr = ';'.join([ word.lemma_ for word in doc if word.pos_ == 'VERB' and not word.is_stop])
        newdf.at[erow, 'Verbs'] = verbstr
        
        nnstr = ';'.join([ word.text for word in doc if word.pos_ == 'NOUN' and not word.is_stop])
        newdf.at[erow, 'Nouns'] = nnstr

        namestr = ';'.join([ ent.text for ent in doc.ents if ent.label_ == 'PERSON' and len(ent.text.split(' ')) > 1 ])
        newdf.at[erow, 'Names'] = namestr
        
        orgstr = ';'.join([ ent.text for ent in doc.ents if ent.label_ == 'ORG' ])
        newdf.at[erow, 'Organizations'] = orgstr
        


    #print(newdf)
    newdf.to_csv(''.join(['Extracted Names ',f,'.csv']))
    #print('done!')

File A-B 0
File C-H 0
File I-N 0
File O-Z 0
